## Read subreddits Posts

In [ ]:
import glob

json_files = glob.glob('./data/*.json')
print(json_files)

import json

def getRedditSubreddits(json_files):
  subreddits = {}
  for json_file in json_files:
    with open(json_file) as f:
      data = json.load(f)
      firstKey = next(iter(data))
      subredditName = data[firstKey]['subreddit']
      subreddits[subredditName] = data
  return subreddits

def saveredditSubreddits(subreddits):
  for subreddit in subreddits:
    with open('./data/' + subreddit + '_top_posts.json', 'w') as outfile:
      json.dump(subreddits[subreddit], outfile, indent=4)

subreddits = getRedditSubreddits(json_files)

print("subreddits", subreddits.keys())


In [22]:
import requests
import json

mediaDomains = ['i.redd.it', 'v.redd.it', 'i.imgur.com', 'youtu.be', 'youtube.com', 'i.reddituploads.com']

def generateFromApi(prompt):
  url = "http://127.0.0.1:5000/api/v1/generate"
  data = {"prompt": prompt}
  headers = {"Content-Type": "application/json"}
  response = requests.post(url, data=json.dumps(data), headers=headers)
  generated_text = response.json()['results'][0]['text']
  generated_text = generated_text.replace(prompt, '') #remove prompt
  generated_text = generated_text.strip() #remove leading and trailing whitespace
  #remove leading newlines
  while generated_text[0] == "\n":
    generated_text = generated_text[1:]
  return generated_text

def promptFromRedditPost(post):
  return "You are a professional writer. Write an article."  + "\nURL: " + post['url']  + "\nTitle: " + post['title'] + "\nArticle text: "

def isMediaDomain(url):
  for mediaDomain in mediaDomains:
    if mediaDomain in url:
      return True
  return False

In [23]:
for subreddit_name in subreddits:
  subreddit = subreddits[subreddit_name]
  print("subreddit", subreddit_name)
  for post_id in subreddit:
    post = subreddit[post_id]
    isValidUrlPost = post['url'] != None and post['text'] == None and not isMediaDomain(post['url'])
    if not isValidUrlPost:
      continue
    # if the post text is null and there's a url continue
    prompt = promptFromRedditPost(post)
    generatedText = generateFromApi(prompt)
    subreddit[post_id]['text'] = generatedText # set the text to the generated text
    print(subreddit[post_id])


subreddit facepalm
subreddit announcements
subreddit DunderMifflin
subreddit wholesomememes
subreddit science
{'author_fullname': 't2_6hji', 'author_name': 'mvea', 'created_utc': 1607978957.0, 'domain': 'aftau.org', 'gilded': 0, 'gildings': {}, 'id': 'kd67rx', 'is_created_from_ads_ui': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_self': False, 'is_video': False, 'link_flair_text': 'Medicine', 'media_only': False, 'num_comments': 3509, 'over_18': False, 'permalink': '/r/science/comments/kd67rx/led_lights_found_to_kill_coronavirus_efficiently/', 'score': 114429, 'subreddit': 'science', 'subreddit_id': 'mouw', 'text': 'In a groundbreaking study, researchers have discovered that ultraviolet (UV) light emitting diodes (LEDs) can effectively kill the deadly coronavirus, known as SARS-CoV-2. The finding could be a game changer in the fight against COVID-19, as it suggests that UV-LEDs can be installed in air conditioning and water systems to disinfect the air and 

In [25]:

saveredditSubreddits(subreddits)